In [2]:
%pip install ipywidgets cartopy geopandas contextily matplotlib

  Using cached Cartopy-0.21.1.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached pyshp-2.3.1-py2.py3-none-any.whl.metadata (55 kB)
  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 25.8 MB/s eta 0:00:00a 0:00:01
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for cartopy (pyproject.toml) did not run successful

In [10]:
import ipywidgets as widgets
from IPython.display import display
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import geopandas as gpd
import contextily as ctx
import shap
import matplotlib.pyplot as plt
import numpy as np

In [13]:
path = "/Users/I566184/bachelorarbeit_coding/mesogeos/csv/shap_map_csv"
file = "shap_map_rf.csv"

In [19]:
df = pd.read_csv(f"{path}/{file}")
df.rename(columns={"x": "lon", "y": "lat"}, inplace=True)
df_pos = df[df["label"] == 1] #rename to df to only show positive labels
df_neg = df[df["label"] == 0] #rename to df to only show negative labels

feature_names = [col for col in df.columns if col not in ["lon", "lat"]]
dropdown = widgets.Dropdown(
    options=feature_names,
    value=feature_names[0],
    description='SHAP Feature:'
)

dataset_selector = widgets.Dropdown(
    options=["all", "positive", "negative"],
    value="all",
    description="Label filter:"
)


def plot_feature_map(feature_name, label_filter):
    if label_filter == "positive":
        data = df_pos
    elif label_filter == "negative":
        data = df_neg
    else:
        data = df
    
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Set extent basierend auf Longitude (x) und Latitude (y)
    ax.set_extent([
        data["lon"].min() - 1, data["lon"].max() + 1,
        data["lat"].min() - 1, data["lat"].max() + 1
    ], crs=ccrs.PlateCarree())

    # Basemap
    ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor='#f0e9dc')
    ax.add_feature(cfeature.OCEAN.with_scale('50m'), facecolor='#cce6ff')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.6)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.6)
    ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='lightblue', edgecolor='k', linewidth=0.3)
    ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='blue', linewidth=0.2)

    # Scatterplot
    vmax = np.abs(data[feature_name]).max()
    vmin = -vmax
    
    sc = ax.scatter(
        data["lon"], data["lat"],
        c=data[feature_name],
        cmap='RdBu_r',         # Rot-Blau umgekehrt
        vmin=vmin, vmax=vmax,  # Symmetrische Farbskala
        s=5, alpha=0.8,
        transform=ccrs.PlateCarree()
    )

    # Colorbar
    cb = plt.colorbar(sc, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
    cb.set_label(f'SHAP: {feature_name}')

    plt.title(f"SHAP Values for '{feature_name}' - MLP")
    plt.show()
widgets.interact(plot_feature_map, feature_name=dropdown, label_filter=dataset_selector);


interactive(children=(Dropdown(description='SHAP Feature:', options=('d2m', 'dem', 'lai', 'lc_agriculture', 'l…

In [17]:
print(df_pos.shape)

(1369, 27)


In [2]:
def plot_shap_geopandas(df, shap_column, title="SHAP Map", cmap="viridis"):
    """
    df: Pandas DataFrame mit Spalten 'lat', 'lon' und dem SHAP-Feature
    shap_column: Name der Spalte mit SHAP-Werten (z. B. 'wind_speed')
    """
    # Erstelle GeoDataFrame mit Punkten aus lat/lon
    gdf = gpd.GeoDataFrame(df,
                           geometry=gpd.points_from_xy(df['lon'], df['lat']),
                           crs="EPSG:4326")  # WGS84
    # Reprojiziere in Web-Mercator (für Hintergrundkarten wie OpenStreetMap)
    gdf = gdf.to_crs(epsg=3857)

    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, column=shap_column, cmap=cmap, legend=True, markersize=10, alpha=0.8)

    ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatter)

    ax.set_axis_off()
    ax.set_title(f"SHAP Values for {shap_column}", fontsize=14)
    plt.tight_layout()
    plt.show()


In [3]:
df = pd.read_csv(f"{path}/{file}")
df.rename(columns={"x": "lon", "y": "lat"}, inplace=True)
plot_shap_geopandas(df, shap_column="wind_speed", title="SHAP für wind_speed")

NameError: name 'path' is not defined